<a href="https://colab.research.google.com/github/ashleyrennnnnn/Assignments-Machine-Learning/blob/main/Regression_model_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb


In [89]:
train_data = pd.read_csv("train-data.csv", index_col="gwb_code_10")
test_data = pd.read_csv("test-data.csv", index_col="gwb_code_10")


In [90]:
X = train_data.drop(["loneliness", "sporting"], axis=1)
y_loneliness = train_data["loneliness"]
y_sporting = train_data["sporting"]

X_test = test_data.drop(["loneliness", "sporting"], axis=1)
y_test_loneliness = test_data["loneliness"]
y_test_sporting = test_data["sporting"]


## Regression model


In [91]:
# The imputer that will be used in all models:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor

imputer = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=10), max_iter=20, tol=1e-3, random_state=100)
X_pre_imputed = imputer.fit_transform(X)
X_test_pre_imputed = imputer.transform(X_test)

In [92]:
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from typing import Union
from sklearn.model_selection import GridSearchCV, train_test_split
from typing import Union, List, Dict
from sklearn.metrics import mean_absolute_error, r2_score


def grid_search_and_test(model: Pipeline, params: Dict[str, List[float]], target_variable: str = "loneliness", pre_imputed: bool = False) -> pd.DataFrame:
    gsc = GridSearchCV(model, param_grid=params, n_jobs=3, error_score=0.0, verbose=3)

    if target_variable == "loneliness":
        y = y_loneliness
        y_test = y_test_loneliness
    elif target_variable == "sporting":
        y = y_sporting
        y_test = y_test_sporting
    else:
        raise ValueError("Wrong Target variable")

    if pre_imputed:
        X_grid = X_pre_imputed
        X_test_grid = X_test_pre_imputed
    else:
        X_grid = X
        X_test_grid = X_test

    gsc.fit(X_grid, y)
    print("Best parameters found for the model:", gsc.best_params_)
    model.set_params(**gsc.best_params_)
    model.fit(X_grid, y)

    pred = model.predict(X_test_grid)
    scores = {
        "Mean Absolute Error": [mean_absolute_error(y_test, pred)],
        "R2 score": [r2_score(y_test, pred)],
    }
    # Use this to see the top 5 results
    # display(pd.DataFrame(gsc.cv_results_).sort_values(by="rank_test_score").head(5))
    return pd.DataFrame(scores)


For Loneliness


Ridge，Lasso，LinearRegression

In [93]:
# Thus the only thing that is needed to test all the models is to create a pipeline like this:
ridge_pipe = Pipeline([("scaler", StandardScaler()),("estimator", Ridge(random_state=100))])

# Then we need to create the "grid" with paramaters that we want to search through, so here different alphas
params = {"estimator__alpha": np.logspace(-5,1,num=50)}

# We can now call the function we defined above to get the df with the test results, and the best parameter
# Make sure that you don't get a ridicoulus amounts of totalling fits, i.e > 1 000 as that might take way to much time!
# Use pre_imputed = True to significantlly increase the time to run it. However, a small data leakage occurs then, but it is okay!
ridge_result = grid_search_and_test(model=ridge_pipe, params=params, target_variable="loneliness", pre_imputed=True)
ridge_result

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found for the model: {'estimator__alpha': 0.00029470517025518097}


,Mean Absolute Error,R2 score
0,1.908699,0.76846


In [94]:
lasso_pipe = Pipeline([("scaler", StandardScaler()),("estimator", Lasso(random_state=100))])
params = {"estimator__alpha": np.logspace(-5,1,num=50)}
lasso_result = grid_search_and_test(model=lasso_pipe, params=params, target_variable="loneliness", pre_imputed=True)
lasso_result


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found for the model: {'estimator__alpha': 0.04714866363457394}


,Mean Absolute Error,R2 score
0,1.980164,0.781263


In [95]:
lr_pipe = Pipeline([("scaler", StandardScaler()),("estimator", LinearRegression(n_jobs=-1))])
params = { }
lr_result = grid_search_and_test(model=lr_pipe, params=params, target_variable="loneliness", pre_imputed=True)
lr_result

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters found for the model: {}


,Mean Absolute Error,R2 score
0,1.909054,0.768639


Desicion Tree Classifier and Random forest Classifier

In [96]:
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from typing import List

In [102]:
tree = Pipeline([("scaler", StandardScaler()),("estimator", DecisionTreeRegressor())])
params = {"estimator__max_depth": np.linspace(2, 10, num=5,dtype=int),
          "estimator__max_leaf_nodes":np.linspace(2,20,num=5,dtype=int)}
tree_result = grid_search_and_test(model=tree, params=params, target_variable="loneliness", pre_imputed=True)
tree_result

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters found for the model: {'estimator__max_depth': 10, 'estimator__max_leaf_nodes': 20}


,Mean Absolute Error,R2 score
0,2.186012,0.778141


In [103]:
forest = Pipeline([("scaler", StandardScaler()),("estimator", RandomForestRegressor())])
params = {"estimator__max_depth": np.linspace(2, 30, num=5,dtype=int),
          "estimator__max_leaf_nodes":np.linspace(2,50,num=5,dtype=int)}
forest_result = grid_search_and_test(model=forest, params=params, target_variable="loneliness", pre_imputed=True)
forest_result

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters found for the model: {'estimator__max_depth': 30, 'estimator__max_leaf_nodes': 50}


,Mean Absolute Error,R2 score
0,1.807901,0.845932


In [105]:
# DataFrame using arrays.
import pandas as pd
 
# initialize data of lists.
data = {'Ridge':[1.908699,0.76846],
        'Lasso':[1.980164,0.781263],
        'LinearRegression': [1.909054,0.768639],
        'Decision Tree': [2.186012,0.778141],
        'Random forest':[1.807901,0.845932],}
 
# Creates pandas DataFrame.
df = pd.DataFrame(data, index=['Mean absolute error',
                               'r2 score'])
print("The best scores found for loneliness:")
# print the data
df

The best scores found for loneliness:


,Ridge,Lasso,LinearRegression,Decision Tree,Random forest
Mean absolute error,1.908699,1.980164,1.909054,2.186012,1.807901
r2 score,0.768460,0.781263,0.768639,0.778141,0.845932


For sport

Ridge,Lasso,Linear Regression

In [108]:
ridge_pipe = Pipeline([("scaler", StandardScaler()),("estimator", Ridge(random_state=100))])
params = {"estimator__alpha": np.logspace(-5,1,num=50)}

ridge_result = grid_search_and_test(model=ridge_pipe, params=params, target_variable="sporting", pre_imputed=True)
ridge_result

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found for the model: {'estimator__alpha': 10.0}


,Mean Absolute Error,R2 score
0,3.652635,0.586418


In [109]:
lasso_pipe = Pipeline([("scaler", StandardScaler()),("estimator", Lasso(random_state=100))])
params = {"estimator__alpha": np.logspace(-5,1,num=50)}
lasso_result = grid_search_and_test(model=lasso_pipe, params=params, target_variable="sporting", pre_imputed=True)
lasso_result

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found for the model: {'estimator__alpha': 0.08286427728546843}


,Mean Absolute Error,R2 score
0,3.783424,0.580497


In [115]:
lasso_pipe.set_params(estimator__alpha=0.04714866363457394)
lasso_pipe.fit(X_pre_imputed, y_loneliness)
loneliness = pd.DataFrame({name: [lasso_pipe["estimator"].coef_[idx]]for idx, name in enumerate(X.columns)}).transpose().rename(columns={0: "loneliness"})

lasso_pipe.set_params(estimator__alpha=0.08286427728546843)
lasso_pipe.fit(X_pre_imputed, y_sporting)
sporting= pd.DataFrame({name: [lasso_pipe["estimator"].coef_[idx]]for idx, name in enumerate(X.columns)}).transpose().rename(columns={0: "sporting"})
pd.concat([sporting, loneliness],axis=1).replace(0, np.nan).dropna(thresh=2)



,sporting,loneliness
a_marok,-0.157233,0.139387
a_tur,-0.304386,0.411997
p_geb,0.009810,-0.108109
p_ste,-0.817447,-0.045022
g_hhgro,-0.177835,-1.524670
g_woz,3.256600,-1.074731
p_bewndw,0.667576,-0.260413
p_wcorpw,-2.966450,1.449327
p_e_o_w,-0.033501,0.066002
p_bjj2k,-0.976350,0.206644


In [111]:
lr_pipe = Pipeline([("scaler", StandardScaler()),("estimator", LinearRegression(n_jobs=-1))])
params = {}
lr_result = grid_search_and_test(model=lr_pipe, params=params, target_variable="sporting", pre_imputed=True)
lr_result

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters found for the model: {}


,Mean Absolute Error,R2 score
0,3.620344,0.581247


Desicion Tree Classifier and Random forest Classifier

In [112]:
tree = Pipeline([("scaler", StandardScaler()),("estimator", DecisionTreeRegressor())])
params = {"estimator__max_depth": np.linspace(2, 30, num=5,dtype=int),
          "estimator__max_leaf_nodes":np.linspace(2,50,num=5,dtype=int)}
tree_result = grid_search_and_test(model=tree, params=params, target_variable="sporting", pre_imputed=True)
tree_result

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters found for the model: {'estimator__max_depth': 9, 'estimator__max_leaf_nodes': 50}


,Mean Absolute Error,R2 score
0,3.628099,0.623321


In [118]:
forest = Pipeline([("scaler", StandardScaler()),("estimator", RandomForestRegressor(max_depth=5))])
params = {"estimator__max_depth": np.linspace(2, 30, num=5,dtype=int),
          "estimator__max_leaf_nodes":np.linspace(2,50,num=5,dtype=int)}
forest_result = grid_search_and_test(model=forest, params=params, target_variable="sporting", pre_imputed=True)
forest_result

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters found for the model: {'estimator__max_depth': 30, 'estimator__max_leaf_nodes': 50}


,Mean Absolute Error,R2 score
0,3.310625,0.686475


In [119]:
# DataFrame using arrays.
import pandas as pd
 
# initialize data of lists.
data = {'Ridge':[3.652635,0.586418],
        'Lasso':[3.783424,0.580497],
        'LinearRegression': [3.620344,0.581247],
        'Decision Tree': [3.628099,0.623321],
        'Random forest':[3.310625	,0.686475]}
 
# Creates pandas DataFrame.
df = pd.DataFrame(data, index=['Mean absolute error',
                               'r2 score'])
print("The best scores found for sporting:")
# print the data
df

The best scores found for sporting:


,Ridge,Lasso,LinearRegression,Decision Tree,Random forest
Mean absolute error,3.652635,3.783424,3.620344,3.628099,3.310625
r2 score,0.586418,0.580497,0.581247,0.623321,0.686475
